#### Names of people in the group

Please write the names of the people in your group in the next cell.

Anne Torgersen

Aaryan Neupane

In [ ]:
# We need to install 'ipython_unittest' to run unittests in a Jupyter notebook
!pip install -q ipython_unittest

In [ ]:
# Loading modules that we need
import unittest
from pyspark.sql.dataframe import DataFrame
from typing import Any

In [ ]:
# A helper function to load a table (stored in Parquet format) from DBFS as a Spark DataFrame 
def load_df(table_name: "name of the table to load") -> DataFrame:
    return spark.read.parquet(table_name)

users_df = load_df("/user/hive/warehouse/users")
comments_df = load_df("/user/hive/warehouse/comments")
posts_df = load_df("/user/hive/warehouse/posts")

#### Subtask 1: implenenting two helper functions
Impelment these two functions:
1. 'run_query' that gets a Spark SQL query and run it on df which is a Spark DataFrame; it returns the content of the first column of the first row of the DataFrame that is the output of the query;
2. 'run_query2' that is similar to 'run_query' but instead of one DataFrame gets two; it returns the content of the first column of the first row of the DataFrame that is the output of the query.

Note that the result of a Spark SQL query is itself a Spark DataFrame.

In [ ]:
def run_query(query: "a SQL query string", df: "the DataFrame that the query will be executed on") -> Any:
    df.createOrReplaceTempView("df")
    result_df = spark.sql(query)
    first_row = result_df.first()
    return first_row[0]

def run_query2(query: "a SQL query string", df1: "DataFrame A", df2: "DataFrame B") -> Any:
    df1.createOrReplaceTempView("df1")
    df2.createOrReplaceTempView("df2")
    result_df = spark.sql(query)
    first_row = result_df.first()
    return first_row[0]

In [ ]:
# Loading 'ipython_unittest' so we can use '%%unittest_main' magic command
%load_ext ipython_unittest

#### Subtask 2: writing a few queries
Write the following queries in SQL to be executed by Spark in the next cell.

1. 'q1': find the 'Id' of the most recently created post ('df' is 'posts_df') 
2. 'q2': find the number users
3. 'q3': find the 'Id' of the user who posted most number of answers
4. 'q4': find the number of questions
5. 'q5': find the display name of the user who posted most number of comments

Note that 'q1' is already available below as an example. Moreover, remmebr that Spark supports ANSI SQL 2003 so your queries have to comply with that standard.

In [ ]:
q1 = '''
SELECT * 
FROM df 
ORDER BY CreationDate
DESC limit 1
'''

q2 = '''
SELECT count(*) 
FROM df '''

q3 = '''
SELECT OwnerUserId FROM df
GROUP BY OwnerUserId
ORDER BY count(CASE WHEN PostTypeId = 2 THEN 1 ELSE NULL END)
DESC limit 1
'''

q4 = '''
SELECT count(*) 
FROM df 
WHERE PostTypeId == 1
'''

q5 = '''
SELECT df1.DisplayName 
FROM df1 INNER JOIN df2 
ON df1.Id = df2.UserId 
GROUP BY df1.Id, df1.DisplayName 
ORDER BY count(*) 
DESC limit 1
'''

#### Subtask 3: validating the implementations by running the tests

Run the cell below and make sure that all the tests run successfully.

In [ ]:
%%unittest_main
class TestTask2(unittest.TestCase):
    def test_q1(self):
        # find the id of the most recent post
        r = run_query(q1, posts_df)
        self.assertEqual(r, 95045)

    def test_q2(self):
        # find the number of the users
        r = run_query(q2, users_df)
        self.assertEqual(r, 91616)

    def test_q3(self):
        # find the user id of the user who posted most number of answers
        r = run_query(q3, posts_df)
        self.assertEqual(r, 64377)

    def test_q4(self):
        # find the number of questions
        r = run_query(q4, posts_df)
        self.assertEqual(r, 28950)

    def test_q5(self):
        # find the display name of the user who posted most number of comments
        r = run_query2(q5, users_df, comments_df)
        self.assertEqual(r, "Neil Slater")

Success

.....
----------------------------------------------------------------------
Ran 5 tests in 11.289s

OK
Out[7]: <unittest.runner.TextTestResult run=5 errors=0 failures=0>

#### Subtask 4: answering to questions about Spark related concepts

Please answer the following questions. Write your answer in one to two short paragraphs. Don't copy-paste; instead, write your own understanding.

1. What is the difference between 'DataFrame', 'Dataset', and 'Resilient Distributed Datasets (RDD)'? 
2. When do you suggest using RDDs instead of using DataFrames?
3. What is the main benefit of using DataSets instead of DataFrames?

Write your answers in the next cell.

1. What is the difference between 'DataFrame', 'Dataset', and 'Resilient Distributed Datasets (RDD)'?

- These terms refers to different data structures of Spark. All of these concepts are used for distributed processing of data, and they are all in use today. The RDDs came first when Spark launched in 2011, and DataFrames and Datasets have arrived later. The RDD is the fundamental data structure of spark, while DataFrames and DataSets were developed to provide more structured, optimized and user friendly ways of working with distributed data. The data in DataFrames is organized into named columns, and in difference to RDDs allows developers to debug during runtime.  Datasets are an extension of DataFrames that provide a type-safe, object-oriented programming interface. The coding is similar to RDDs, but faster. 

2. When do you suggest using RDDs instead of using DataFrames?

- RDDs could be a better choice if your data processing involves complex operations requiring low-level control. Some operations are also specific for RDDs, and can’t be executed in the DataFrame format. DataFrames is beneficial when working with structured data, but if the data lacks structure, RDDs could be more suitable. 

3. What is the main benefit of using DataSets instead of DataFrames?

- The main benefit of using DataSets instead of DataFrames is the advantages that comes with ensuring type-safety. This can lead to improved code reliability and performance.